In [1]:
import numpy as np
import json
import os

In [2]:
folders_arithmetic_low_35 = ['arithmetic_low_gpt_3.5', 'arithmetic_low_gpt_3.5_1']
folders_arithmetic_high_35 = ['arithmetic_high_gpt_3.5', 'arithmetic_high_gpt_3.5_1']
folders_puzzle_35 = ['puzzle_low_gpt_3.5', 'puzzle_low_gpt_3.5_1']

folders_arithmetic_low_4 = ['arithmetic_low_gpt_4', 'arithmetic_low_gpt_4_1']
folders_arithmetic_high_4 = ['arithmetic_high_gpt_4', 'arithmetic_high_gpt_4_1']
folders_puzzle_4 = ['puzzle_low_gpt_4', 'puzzle_low_gpt_4_1']

In [3]:
folders = [
    folders_arithmetic_low_35, folders_arithmetic_high_35,
    folders_puzzle_35, folders_arithmetic_low_4,
    folders_arithmetic_high_4, folders_puzzle_4
]

In [4]:
out_names = [
    'arithmetic_low_gpt_3.5_avg', 'arithmetic_high_gpt_3.5_avg',
    'puzzle_gpt_3.5_avg', 'arithmetic_low_gpt_4_avg',
    'arithmetic_high_gpt_4_avg', 'puzzle_gpt_4_avg'
]

In [5]:
with open('arithmetic_high_gpt_3.5/results.json', 'r') as f:
    file = json.loads(f.read())
print(file[0])

{'name': 'default_context', 'no_context': {'abs_edit_distance': {'mean': 1.19, 'std': 1.4191194452899307}, 'rel_edit_distance': {'mean': 0.1523214285714286, 'std': 0.18079781736875625}, 'abs_distance': {'mean': 6587.8, 'std': 11605.235678778787}, 'rel_distance': {'mean': 0.000242914909737679, 'std': 0.0003641732450016474}, 'accuracy': {'mean': 0.55, 'std': 0.4974937185533099}}, 'context': {'abs_edit_distance': {'mean': 1.23, 'std': 1.4550257729676133}, 'rel_edit_distance': {'mean': 0.15750000000000003, 'std': 0.18534269142673557}, 'abs_distance': {'mean': 6682.8, 'std': 12547.525977657911}, 'rel_distance': {'mean': 0.00023745184920375342, 'std': 0.00036359896550056787}, 'accuracy': {'mean': 0.55, 'std': 0.4974937185533099}}, 'extraction_rate': 1.0}


In [6]:
def average_metrics(a, b, a_rate, b_rate):
    n_runs = 100
    n = n_runs * a_rate
    m = n_runs * b_rate
    obj = {}
    metrics = ['abs_edit_distance', 'rel_edit_distance', 
               'abs_distance', 'rel_distance',
               'accuracy']
        
    for metric in metrics:
        a_mean = a[metric]['mean']
        b_mean = b[metric]['mean']
        
        a_std = a[metric]['std']
        b_std = b[metric]['std']
        n = 100
        combined_var = ((n * (a_std**2) + m * (b_std**2)) / (n + m)) + \
                  (n * m * ((a_mean - b_mean) ** 2) / ((n + m) ** 2))
        
        obj[metric] = {
            'mean': (a_mean * n + b_mean * m) / (n+m),
            'std': np.sqrt(combined_var)
        }
    
    return obj

In [8]:
for files, out in zip(folders, out_names):
    with open(f'{files[0]}/results.json', 'r') as f:
        file_0 = json.loads(f.read())
    with open(f'{files[1]}/results.json', 'r') as f:
        file_1 = json.loads(f.read())
    
    out_arr = []
    
    for a, b in zip(file_0, file_1):
        out_obj = {
            'name': a['name'],
            'no_context': average_metrics(a['no_context'], b['no_context'], a['extraction_rate'], b['extraction_rate']),
            'context': average_metrics(a['context'], b['context'], a['extraction_rate'], b['extraction_rate']),
            'extraction_rate': np.mean([a['extraction_rate'], b['extraction_rate']])
        }
        out_arr.append(out_obj)
        
    with open(f'{out}_results_new.json', 'w') as f:
        f.write(json.dumps(out_arr, indent=4))

In [11]:
for name in out_names:
    with open(f'{name}_results.json', 'r') as f:
        file = json.loads(f.read())

    print(name)
    for context in file:
        print(context['name'])
        for c in ['context', 'no_context']:
            print(c)
            data = context[c]
            print(f"{data['abs_edit_distance']['mean']:.3f}" +\
                  f" ({data['abs_edit_distance']['std']:.3f}) & " +\
                  f"{data['rel_edit_distance']['mean'] * 100:.1f}" + \
                  f" ({data['rel_edit_distance']['std'] * 100:.1f}) & " +\
                  f"{data['rel_distance']['mean'] * 100:.1f}" + \
                  f" ({data['rel_distance']['std'] * 100:.1f}) & " +\
                  f"{data['accuracy']['mean']*100:.1f}")

arithmetic_low_gpt_3.5_avg
default_context
context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
no_context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
code_context
context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
no_context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
explain_context
context
0.180 (0.740) & 4.5 (18.5) & 5.5 (22.0) & 94.0
no_context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
impersonation_context
context
0.265 (0.903) & 6.6 (22.6) & 5.3 (21.3) & 91.5
no_context
0.025 (0.211) & 0.6 (5.3) & 0.0 (0.0) & 98.5
restorying_context
context
0.926 (1.382) & 28.4 (41.2) & 23.8 (53.1) & 65.0
no_context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
arithmetic_high_gpt_3.5_avg
default_context
context
1.410 (1.450) & 18.0 (18.5) & 0.0 (0.0) & 47.0
no_context
1.420 (1.461) & 18.1 (18.6) & 0.0 (0.0) & 47.0
code_context
context
1.490 (1.596) & 19.1 (20.3) & 0.0 (0.1) & 49.5
no_context
1.350 (1.410) & 17.6 (18.1) & 0.0 (0.0) & 46.0
explain_context
context
1.565 (2.047) & 20.9 

In [10]:
for name in out_names:
    with open(f'{name}_results_new.json', 'r') as f:
        file = json.loads(f.read())

    print(name)
    for context in file:
        print(context['name'])
        for c in ['context', 'no_context']:
            print(c)
            data = context[c]
            print(f"{data['abs_edit_distance']['mean']:.3f}" +\
                  f" ({data['abs_edit_distance']['std']:.3f}) & " +\
                  f"{data['rel_edit_distance']['mean'] * 100:.1f}" + \
                  f" ({data['rel_edit_distance']['std'] * 100:.1f}) & " +\
                  f"{data['rel_distance']['mean'] * 100:.1f}" + \
                  f" ({data['rel_distance']['std'] * 100:.1f}) & " +\
                  f"{data['accuracy']['mean']*100:.1f}")

arithmetic_low_gpt_3.5_avg
default_context
context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
no_context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
code_context
context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
no_context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
explain_context
context
0.180 (0.740) & 4.5 (18.5) & 5.5 (22.0) & 94.0
no_context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
impersonation_context
context
0.265 (0.903) & 6.6 (22.6) & 5.3 (21.3) & 91.5
no_context
0.025 (0.211) & 0.6 (5.3) & 0.0 (0.0) & 98.5
restorying_context
context
0.926 (1.381) & 28.4 (41.2) & 23.9 (53.6) & 65.0
no_context
0.000 (0.000) & 0.0 (0.0) & 0.0 (0.0) & 100.0
arithmetic_high_gpt_3.5_avg
default_context
context
1.410 (1.450) & 18.0 (18.5) & 0.0 (0.0) & 47.0
no_context
1.420 (1.461) & 18.1 (18.6) & 0.0 (0.0) & 47.0
code_context
context
1.489 (1.595) & 19.1 (20.3) & 0.0 (0.1) & 49.5
no_context
1.349 (1.409) & 17.5 (18.1) & 0.0 (0.0) & 46.0
explain_context
context
1.565 (2.048) & 20.9 

In [14]:
with open('arithmetic_high_gpt_4_1/impersonation_context.json', 'r') as f:
    data = json.loads(f.read())

In [16]:
res = []
for row in data:
    if 'metrics' in row['no_context']:
        res.append(row['no_context']['metrics']['rel_distance'])

In [17]:
np.mean(res)

0.09013361538327079

In [19]:
import matplotlib.pyplot as plt

In [21]:
np.max(res)

8.996928652205202